<a href="https://colab.research.google.com/github/MariaMengozzi/esameDI/blob/main/Progetto_esame_DI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Determinare il prezzo di vendita delle auto

**Programmazione di Applicazioni Data Intensive**\
Laurea in Ingegneria e Scienze Informatiche\
DISI - Università di Bologna, Cesena

Maria Mengozzi

Per prima cosa carichiamo le librerie necessarie ad elaborare i dati:


*   *os.path* per elaborare i percorsi dei file
*   *NumPy* per creare e operare su array N dimesionali
*   *pandas* per caricare e manipolare dati tabulari
*   *matplotlib* per creare grafici, viene inoltre aggiunta il comando per visualizzarli all'interno del notebook.



In [21]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Descrizione del problema


Carichiamo il dataset da [Kaggle](https://www.kaggle.com/nehalbirla/vehicle-dataset-from-cardekho?select=Car+details+v3.csv) contenente le informazioni sulle auto usate elencate su [www.cardekho.com](www.cardekho.com). 

L' obiettivo del progetto è quello di predire una variabile continua, ossia il prezzo di vendita di un' auto usata in base alle sue caratteristiche.

In [26]:
file_zip_url = "https://github.com/MariaMengozzi/esameDI/raw/main/UsedCarsDataset.zip"
file_zip_name = "UsedCarsDataset.zip"
file_name = 'Car details v3.csv'

if not os.path.exists(file_zip_name):
    from urllib.request import urlretrieve
    urlretrieve(file_zip_url, file_zip_name)
    from zipfile import ZipFile
    with ZipFile(file_zip_name) as f:
        f.extractall()

In [28]:
with open(file_name) as dataFile:
    data_raw = pd.read_csv(dataFile, sep=",")